<a href="https://colab.research.google.com/github/peterbabulik/QuantumWalker/blob/main/QantumWalkers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install cirq

In [ ]:
import cirq
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import time # To measure execution time

# --- Parameters ---
ROWS = 30        # Grid size - adjust as needed
COLS = 30
N_SITES = ROWS * COLS
DEPTH = 100       # Number of steps - Increase later for more overlap
STATE_DIM = 4 * N_SITES

# Starting positions (example: near opposite corners)
START_POS_1 = (1, 1)
START_POS_2 = (ROWS - 2, COLS - 2)

# Coin choice
H = (1/np.sqrt(2)) * np.array([[1, 1], [1, -1]], dtype=np.complex128)
COIN_MATRIX = np.kron(H, H) # H⊗H
COIN_NAME = "Hadamard"

# --- Helper Functions (get_index, get_coords_from_index) ---
# (Keep as before)
def get_index(cx, cy, r, c, rows, cols):
    if not (0 <= cx <= 1 and 0 <= cy <= 1 and 0 <= r < rows and 0 <= c < cols):
        raise ValueError(f"Invalid coords/coin: cx={cx}, cy={cy}, r={r}, c={c} for grid {rows}x{cols}")
    return cx + 2*cy + 4*c + 4*cols*r

def get_coords_from_index(k, rows, cols):
    n_sites = rows*cols
    state_dim = 4*n_sites
    if not 0 <= k < state_dim:
         raise ValueError(f"Index k={k} out of bounds for state_dim={state_dim}")
    cx = k % 2
    cy = (k // 2) % 2
    c = (k // 4) % cols
    r = k // (4 * cols)
    return cx, cy, r, c

# --- Initial State Preparation ---
def prepare_initial_state(initial_state_vector, rows, cols, initial_coords, initial_coin_state=(0,0)):
    r_init, c_init = initial_coords
    cx_init, cy_init = initial_coin_state
    start_index = get_index(cx_init, cy_init, r_init, c_init, rows, cols)
    initial_state_vector[start_index] = 1.0
    # print(f"Initialized walker at ({r_init}, {c_init}) in coin state |{cx_init}{cy_init}>")

# --- Build Unitary Matrix ---
def build_qw_unitaries(rows, cols, coin_matrix):
    n_sites = rows * cols
    state_dim = 4 * n_sites
    I_sites = np.eye(n_sites, dtype=np.complex128)
    C_full = np.kron(I_sites, coin_matrix)
    S_full = np.zeros((state_dim, state_dim), dtype=np.complex128)
    for k_in in range(state_dim):
        cx, cy, r, c = get_coords_from_index(k_in, rows, cols)
        r_new, c_new = r, c
        if cx == 0 and cy == 0: c_new = (c - 1 + cols) % cols  # West
        elif cx == 1 and cy == 0: c_new = (c + 1) % cols       # East
        elif cx == 0 and cy == 1: r_new = (r - 1 + rows) % rows # North
        elif cx == 1 and cy == 1: r_new = (r + 1) % rows       # South
        k_out = get_index(cx, cy, r_new, c_new, rows, cols)
        S_full[k_out, k_in] = 1.0
    U_step = S_full @ C_full
    # print(f"Step Unitary U_step shape: {U_step.shape}")
    return U_step

# --- Calculate Position Probability ---
def calculate_position_probability(state_vector, rows, cols):
    num_sites = rows * cols
    state_dim = 4 * num_sites
    if len(state_vector) != state_dim:
         raise ValueError(f"State vector length {len(state_vector)} does not match expected {state_dim}")
    prob_dist_2d = np.zeros((rows, cols))
    for r in range(rows):
        for c in range(cols):
            prob_site = 0.0
            for cx in range(2):
                for cy in range(2):
                    idx = get_index(cx, cy, r, c, rows, cols)
                    amp = state_vector[idx]
                    prob_site += np.abs(amp)**2
            prob_dist_2d[r, c] = prob_site
    return prob_dist_2d

# --- Single Walker Simulation Function ---
def run_single_walker(rows, cols, depth, initial_coords, coin_matrix):
    """Simulates one walker and returns its final probability distribution."""
    n_sites = rows * cols
    state_dim = 4 * n_sites

    initial_state = np.zeros(state_dim, dtype=np.complex128)
    prepare_initial_state(initial_state, rows, cols, initial_coords)

    U_step = build_qw_unitaries(rows, cols, coin_matrix)

    current_state = initial_state.copy()
    # print(f"Running single walker from {initial_coords} for {depth} steps...")
    for step in range(depth):
        current_state = U_step @ current_state
        # Optional normalization
        norm = np.linalg.norm(current_state)
        if np.abs(norm - 1.0) > 1e-6: current_state /= norm
        # if (step + 1) % 10 == 0: print(f"  Walker from {initial_coords}, step {step + 1}/{depth}")

    final_prob_dist = calculate_position_probability(current_state, rows, cols)
    print(f"Finished walker from {initial_coords}.")
    return final_prob_dist

# --- Plotting Function ---
def plot_collision_results(prob1, prob2, prob_overlap, rows, cols, depth, start1, start2, coin_name):
    fig, axs = plt.subplots(1, 3, figsize=(21, 7)) # 3 plots side-by-side
    fig.suptitle(f"Independent 2D QW 'Collision' ({coin_name} Coin, {rows}x{cols}, {depth} Steps)", fontsize=16)

    common_norm = colors.LogNorm(vmin=1e-6, vmax=max(prob1.max(), prob2.max(), 1e-5)) # Log scale, common limits

    # Plot Walker 1
    im1 = axs[0].imshow(prob1, cmap='viridis', norm=common_norm, origin='upper', interpolation='nearest')
    axs[0].set_title(f"Walker 1 (Start: {start1})")
    axs[0].plot(start1[1], start1[0], 'r+', markersize=12)
    axs[0].set_xlabel("Column"); axs[0].set_ylabel("Row")
    axs[0].set_xticks(np.linspace(0, cols-1, min(cols, 6), dtype=int))
    axs[0].set_yticks(np.linspace(0, rows-1, min(rows, 6), dtype=int))
    plt.colorbar(im1, ax=axs[0], label='Probability P(r, c) (log)', fraction=0.046, pad=0.04)

    # Plot Walker 2
    im2 = axs[1].imshow(prob2, cmap='viridis', norm=common_norm, origin='upper', interpolation='nearest')
    axs[1].set_title(f"Walker 2 (Start: {start2})")
    axs[1].plot(start2[1], start2[0], 'r+', markersize=12)
    axs[1].set_xlabel("Column"); axs[1].set_ylabel("Row")
    axs[1].set_xticks(np.linspace(0, cols-1, min(cols, 6), dtype=int))
    axs[1].set_yticks(np.linspace(0, rows-1, min(rows, 6), dtype=int))
    plt.colorbar(im2, ax=axs[1], label='Probability P(r, c) (log)', fraction=0.046, pad=0.04)

    # Plot Overlap
    max_overlap = prob_overlap.max()
    if max_overlap < 1e-9: max_overlap = 1e-9 # Avoid zero range
    norm_overlap = colors.Normalize(vmin=0, vmax=max_overlap) # Linear scale for overlap
    im3 = axs[2].imshow(prob_overlap, cmap='inferno', norm=norm_overlap, origin='upper', interpolation='nearest')
    axs[2].set_title("Overlap Probability (P1 * P2)")
    axs[2].plot(start1[1], start1[0], 'w+', markersize=12, alpha=0.7) # Mark starts faintly
    axs[2].plot(start2[1], start2[0], 'w+', markersize=12, alpha=0.7)
    axs[2].set_xlabel("Column"); axs[2].set_ylabel("Row")
    axs[2].set_xticks(np.linspace(0, cols-1, min(cols, 6), dtype=int))
    axs[2].set_yticks(np.linspace(0, rows-1, min(rows, 6), dtype=int))
    plt.colorbar(im3, ax=axs[2], label='Overlap Prob (P1*P2)', fraction=0.046, pad=0.04)

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

# --- Main Execution ---
if __name__ == "__main__":

    print(f"Simulating two independent walkers on a {ROWS}x{COLS} grid.")
    print(f"Walker 1 start: {START_POS_1}")
    print(f"Walker 2 start: {START_POS_2}")
    print(f"Coin: {COIN_NAME}, Depth: {DEPTH}")

    start_time = time.time()

    # Simulate Walker 1
    prob_dist_1 = run_single_walker(
        rows=ROWS, cols=COLS, depth=DEPTH,
        initial_coords=START_POS_1, coin_matrix=COIN_MATRIX
    )

    # Simulate Walker 2
    prob_dist_2 = run_single_walker(
        rows=ROWS, cols=COLS, depth=DEPTH,
        initial_coords=START_POS_2, coin_matrix=COIN_MATRIX
    )

    end_time = time.time()
    print(f"\nTotal simulation time: {end_time - start_time:.2f} seconds")

    # Calculate Overlap
    prob_overlap = prob_dist_1 * prob_dist_2
    total_overlap = np.sum(prob_overlap)
    print(f"Total overlap probability (Sum P1*P2): {total_overlap:.6e}")

    # Plot results
    plot_collision_results(
        prob_dist_1, prob_dist_2, prob_overlap,
        ROWS, COLS, DEPTH,
        START_POS_1, START_POS_2, COIN_NAME
    )

    print("\nCollision Simulation Finished.")